In [1]:
### EDA Pkgs
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Data Viz Pkg
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Dataset
df = pd.read_csv("covid19_tweets.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'covid19_tweets.csv'

In [ ]:
# Preview
df.head()

In [ ]:
# Check Columns
df.columns

In [ ]:
# Datatype
df.dtypes

In [ ]:
# Source/ Value Count/Distribution of the Sources
df['source'].unique()

In [ ]:
# Source/ Value Count/Distribution of the Sources
df['source'].value_counts()

In [ ]:
# Plot the top value_counts
df['source'].value_counts().nlargest(30)

In [ ]:
# Plot the top value_counts
df['source'].value_counts().nlargest(30).plot(kind='bar')

In [ ]:
# Plot the top value_counts
plt.figure(figsize=(20,10))
df['source'].value_counts().nlargest(30).plot(kind='bar')
plt.xticks(rotation=45)
plt.show()

#### Text Analysis of tweet

In [ ]:
pip install neattext

In [ ]:
# Load Text Cleaning Package
import neattext.functions as nfx

In [ ]:
# Methods/Attrib
dir(nfx)

In [ ]:
df['text'].iloc[2]

### Noise
+ remove mentions/userhandles
+ remove hashtags
+ urls
+ emojis
+ special char

In [ ]:
df.head()

In [ ]:
df['text'].apply(nfx.extract_hashtags)

In [ ]:
df['extracted_hashtags'] = df['text'].apply(nfx.extract_hashtags)

In [ ]:
df[['extracted_hashtags','hashtags']]

In [ ]:
# Cleaning Text
df['clean_tweet'] = df['text'].apply(nfx.remove_hashtags)

In [ ]:
df[['text','clean_tweet']]

In [ ]:
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))

In [ ]:
df[['text','clean_tweet']]

In [ ]:
df['clean_tweet'].iloc[10]

In [ ]:
# Cleaning Text: Multiple WhiteSpaces
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces)

In [ ]:
df['clean_tweet'].iloc[10]

In [ ]:
# Cleaning Text : Remove urls
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)

In [ ]:
# Cleaning Text: Punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts)

In [ ]:
df[['text','clean_tweet']]

### Sentiment Analysis

In [ ]:
from textblob import TextBlob

In [ ]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 'Positive'
    elif sentiment_polarity < 0:
        sentiment_label = 'Negative'
    else:
        sentiment_label = 'Neutral'
    result = {'polarity':sentiment_polarity,
              'subjectivity':sentiment_subjectivity,
              'sentiment':sentiment_label}
    return result

In [ ]:
# Text
ex1 = df['clean_tweet'].iloc[0]

In [ ]:
get_sentiment(ex1)

In [ ]:
df['sentiment_results'] = df['clean_tweet'].apply(get_sentiment)

In [ ]:
df['sentiment_results']

In [ ]:
df['sentiment_results'].iloc[0]

In [ ]:
pd.json_normalize(df['sentiment_results'].iloc[0])

In [ ]:
df = df.join(pd.json_normalize(df['sentiment_results']))

In [ ]:
df.head()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df['sentiment'].value_counts().plot(kind='bar')

In [ ]:
# Plot with seaborn
sns.countplot(df['sentiment'])

In [ ]:
### Keyword Extraction
+ For Positive and Negative Sentiment
+ General

In [ ]:
positive_tweet = df[df['sentiment'] == 'Positive']['clean_tweet']

In [ ]:
neutral_tweet = df[df['sentiment'] == 'Neutral']['clean_tweet']
negative_tweet = df[df['sentiment'] == 'Negative']['clean_tweet']

In [ ]:
positive_tweet

In [ ]:
# Remove Stopwords and Convert to Tokens
positive_tweet_list = positive_tweet.apply(nfx.remove_stopwords).tolist()

In [ ]:
negative_tweet_list = negative_tweet.apply(nfx.remove_stopwords).tolist()
neutral_tweet_list = neutral_tweet.apply(nfx.remove_stopwords).tolist()

In [ ]:
positive_tweet_list 

In [ ]:
# Tokenization
for line in positive_tweet_list:
#     print(line)
    for token in line.split():
        print(token)

In [ ]:
pos_tokens = [token for line in positive_tweet_list  for token in line.split()]

In [ ]:
neg_tokens = [token for line in negative_tweet_list  for token in line.split()]
neut_tokens = [token for line in neutral_tweet_list  for token in line.split()]

In [ ]:
pos_tokens

In [ ]:
# Get Most Commonest Keywords
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
get_tokens(pos_tokens)

In [ ]:
most_common_pos_words = get_tokens(pos_tokens)
most_common_neg_words = get_tokens(neg_tokens)
most_common_neut_words = get_tokens(neut_tokens)

In [ ]:
# Plot with seaborn
neg_df = pd.DataFrame(most_common_neg_words.items(),columns=['words','scores'])

In [ ]:
neg_df

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='words',y='scores',data=neg_df)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Plot with seaborn
pos_df = pd.DataFrame(most_common_pos_words.items(),columns=['words','scores'])
plt.figure(figsize=(20,10))
sns.barplot(x='words',y='scores',data=pos_df)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Plot with seaborn
neut_df = pd.DataFrame(most_common_neut_words.items(),columns=['words','scores'])
plt.figure(figsize=(20,10))
sns.barplot(x='words',y='scores',data=neut_df)
plt.xticks(rotation=45)
plt.show()


In [ ]:
### Word Cloud
from wordcloud import WordCloud

In [ ]:
def plot_wordcloud(docx):
    plt.figure(figsize=(20,10))
    mywordcloud = WordCloud().generate(docx)
    plt.imshow(mywordcloud,interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
pos_docx = ' '.join(pos_tokens)
neg_docx = ' '.join(neg_tokens)
neu_docx = ' '.join(neut_tokens)

In [ ]:
plot_wordcloud(pos_docx)

In [ ]:
plot_wordcloud(neg_docx)

In [ ]:
plot_wordcloud(neu_docx)

In [ ]:
plot_wordcloud(pos_docx)